In [1]:
print("hello")

hello


## Quickstart: Compare runs, choose a model and deploy it to a Rest API

In this quickstart, you will:
- Run a hyperparameter sweep on a training script

- Compare the results of the runs in the MLflow UI

- Choose the best run and register it as a model

- Deploy the model to a REST API

- Build a container image suitable for deployment to a cloud platform

In [2]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

2025-05-09 13:42:44.483177: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-09 13:42:44.777260: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-09 13:42:44.896857: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746798165.152784    1901 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746798165.228223    1901 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746798165.872232    1901 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [3]:
## load the dataset
data=pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [5]:
train, test = train_test_split(data, test_size=0.25, random_state=42)

In [6]:
X_train = train.drop(['quality'], axis=1).values
y_train = train[['quality']].values.ravel()

X_test = test.drop(['quality'], axis=1).values
y_test = test[['quality']].values.ravel()

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1000)

signature = infer_signature(X_train, y_train)

In [8]:
### ANN Model

import mlflow.tensorflow


def train_model(params, epochs, X_train, y_train, X_val, y_val, X_test, y_test):
    mean = np.mean(X_train, axis=0)
    var = np.var(X_train, axis=0)

    model = keras.Sequential(
        [
            keras.Input([X_train.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(32, activation='relu'),
            keras.layers.Dense(1)
        ]
    )

    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],
        momentum = params["momentum"]
    ), 
    loss="mean_squared_error",
    metrics = [keras.metrics.RootMeanSquaredError()]
    )

    with mlflow.start_run(nested=True):
        model.fit(X_train, y_train, validation_data=(X_val, y_val),
                  epochs=epochs,
                  batch_size=64
                  )
        eval_result = model.evaluate(X_val, y_val, batch_size=64)

        eval_rmse = eval_result[1]
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss" : eval_rmse, "status" : STATUS_OK, "model" : model}
    

In [9]:
def objective(params):
    result = train_model(
        params,
        epochs=3,

        X_train= X_train, 
        y_train=y_train,
        X_val=X_val,
        y_val=y_val,
        X_test=X_test,
        y_test=y_test
    )
    return result

In [10]:
space = {
    "lr" : hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum" : hp.uniform("momentum", 0.0, 1.0)
}

In [11]:
import mlflow.tensorflow


mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    trials = Trials()
    best = fmin(
        fn=objective, 
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials= trials
    )

    best_run = sorted(trials.results, key=lambda x : x["loss"])[0]

    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    print(f"Best Parameters: {best}")
    print(f"Bes eval rmse : {best_run["loss"]} ")

2025/05/09 14:07:48 INFO mlflow.tracking.fluent: Experiment with name 'wine-quality' does not exist. Creating a new experiment.


  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

2025-05-09 14:07:48.793546: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 27s 606ms/step - loss: 33.6404 - root_mean_squared_error: 5.8000
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 34.7517 - root_mean_squared_error: 5.8949 - val_loss: 32.3364 - val_root_mean_squared_error: 5.6865

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 32.7824 - root_mean_squared_error: 5.7256
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 30.9083 - root_mean_squared_error: 5.5588 - val_loss: 28.2405 - val_root_mean_squared_error: 5.3142

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 28.1248 - root_mean_squared_error: 5.3033
43/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 26.6821 - root_mean_squared_error: 5.1650 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 26.5974 - root_mean_squared_error: 5.1568 - val_loss: 24.2192 - val_root_mean_squared_error: 4.9213

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s

In [12]:
## Inferencing

from mlflow.models import validate_serving_input

model_uri = 'runs:/1c795b80e8684aec95441a37c7019b0a/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(X_test)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

 1/39 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([[6.235211 ],
       [6.779222 ],
       [6.5871086],
       ...,
       [6.1158543],
       [6.897935 ],
       [5.791879 ]], dtype=float32)

In [13]:
# Load model as a PyFuncModel.
model_uri = 'runs:/1c795b80e8684aec95441a37c7019b0a/model'
loaded_model = mlflow.pyfunc.load_model(model_uri)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(X_test))

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


array([[6.235211 ],
       [6.779222 ],
       [6.5871086],
       ...,
       [6.1158543],
       [6.897935 ],
       [5.791879 ]], dtype=float32)

In [14]:
## Register in the model registry
mlflow.register_model(model_uri,"wine-quality")

Successfully registered model 'wine-quality'.
Created version '1' of model 'wine-quality'.


<ModelVersion: aliases=[], creation_timestamp=1746800125243, current_stage='None', description=None, last_updated_timestamp=1746800125243, name='wine-quality', run_id='1c795b80e8684aec95441a37c7019b0a', run_link=None, source='file:///workspaces/codespaces-blank/1-MLFlow-with-Deep-Learning/mlruns/588539505199625909/1c795b80e8684aec95441a37c7019b0a/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>